# Exemplu de Agent pentru Rezervarea Hotelurilor și Zborurilor

Această soluție te va ajuta să rezervi bilete de avion și hotel. Scenariul este o călătorie de la Londra Heathrow LHR pe 20 februarie 2024 către New York JFK, cu întoarcere pe 27 februarie 2025, zbor la clasa economică doar cu British Airways. Doresc să stau la un hotel Hilton în New York, te rog să furnizezi costurile pentru zbor și hotel.


# Inițializați Serviciul Azure AI Agent și obțineți informații de configurare din **.env**

### **.env** 

Creați un fișier .env 

**.env** conține stringul de conexiune al Serviciului Azure AI Agent, modelul utilizat de AOAI și serviciul corespunzător Google API Search, ENDPOINT, etc.

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "Numele implementării modelului Serviciului Azure AI Agent"

[**NOTE**] Veți avea nevoie de un model cu o limită de 100.000 de tokenuri pe minut și o limită de 600 de cereri pe minut.

  Puteți obține modelul în Azure AI Foundry - Model și Endpoint. 

- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "Stringul de conexiune al proiectului Serviciului Azure AI Agent"

  Puteți obține stringul de conexiune al proiectului în ecranul de prezentare generală al proiectului din portalul AI Foundry.

- **SERPAPI_SEARCH_API_KEY** = "Cheia API pentru SERPAPI Search"
- **SERPAPI_SEARCH_ENDPOINT** = "Endpoint-ul pentru SERPAPI Search"

Pentru a obține Numele Implementării Modelului și Stringul de Conexiune al Proiectului pentru Serviciul Azure AI Agent, trebuie să creați Serviciul Azure AI Agent. Este recomandat să utilizați [acest șablon](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json) pentru a-l crea direct. (***Notă:*** Serviciul Azure AI Agent este în prezent disponibil doar în regiuni limitate. Este recomandat să consultați [acest link](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support) pentru a seta regiunea.)

Agentul trebuie să acceseze SERPAPI. Este recomandat să vă înregistrați utilizând [acest link](https://serpapi.com/searches). După înregistrare, puteți obține o cheie API unică și un ENDPOINT.


# Configurare

Pentru a rula acest notebook, trebuie să te asiguri că ai instalat bibliotecile necesare executând `pip install -r requirements.txt`.


In [ ]:
from semantic_kernel import __version__

__version__

Versiunea ta de Semantic Kernel ar trebui să fie cel puțin 1.27.2.


Încarcă setările și resursele din fișierul tău .env, te rog asigură-te că ai adăugat cheile și setările tale și că ai creat un fișier local .env.


In [ ]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Conectare la Azure

Acum trebuie să te conectezi la Azure. Deschide un terminal și rulează următoarea comandă:

```bash
az login
```

Această comandă îți va solicita să introduci acreditările tale Azure, permițând serviciului Azure AI Agent să funcționeze corect.


# Explicație:
Aceasta este o variabilă care stochează cheia API pentru accesarea unui serviciu API SERP (Search Engine Results Page). O cheie API este un identificator unic utilizat pentru autentificarea cererilor asociate contului tău.

Scop: Scopul acestei linii este de a stoca cheia API într-o variabilă, astfel încât să poată fi utilizată pentru autentificarea cererilor către serviciul API SERP. Cheia API este necesară pentru a accesa serviciul și a efectua căutări.
Cum să obții o cheie API SERP: Pentru a obține o cheie API SERP, urmează pașii generali de la https://serpapi.com (pașii exacti pot varia în funcție de serviciul API SERP specific pe care îl utilizezi):

Alege un serviciu API SERP: Există mai multe servicii API SERP disponibile, cum ar fi SerpAPI, Google Custom Search JSON API și altele. Alege-l pe cel care se potrivește cel mai bine nevoilor tale.

Înregistrează-te pentru un cont: Accesează site-ul serviciului API SERP ales și creează un cont. Este posibil să fie necesar să furnizezi câteva informații de bază și să îți verifici adresa de email.

Creează o cheie API: După ce te-ai înregistrat, conectează-te la contul tău și navighează la secțiunea API sau la tabloul de bord. Caută o opțiune pentru a crea sau genera o nouă cheie API.
Copiază cheia API în fișierul tău .env.


In [ ]:
SERP_API_KEY='SERPAPI_SEARCH_API_KEY'

# Explicație:
BASE_URL: Aceasta este o variabilă care stochează URL-ul de bază pentru endpoint-ul API-ului SERP. Numele variabilei BASE_URL este o convenție utilizată pentru a indica faptul că acest URL este punctul de plecare pentru efectuarea cererilor API.
'https://serpapi.com/search':

Acesta este șirul URL real atribuit variabilei BASE_URL. Reprezintă endpoint-ul pentru efectuarea interogărilor de căutare utilizând API-ul SERP.

# Scop:
Scopul acestei linii este de a defini o constantă care conține URL-ul de bază pentru API-ul SERP. Acest URL va fi utilizat ca punct de plecare pentru construirea cererilor API pentru a efectua operațiuni de căutare.

# Utilizare:
Definind URL-ul de bază într-o variabilă, îl poți reutiliza cu ușurință în codul tău ori de câte ori ai nevoie să faci cereri către API-ul SERP. Acest lucru face codul mai ușor de întreținut și reduce riscul de erori cauzate de hardcodarea URL-ului în mai multe locuri. Exemplul curent este https://serpapi.com/search?engine=bing, care utilizează API-ul de căutare Bing. Diferite API-uri pot fi selectate la https://Serpapi.com


In [ ]:
BASE_URL = 'https://serpapi.com/search?engine=bing'

# Explicație:

Aici se află codul pluginului tău.

Definiția Clasei: `class BookingPlugin`: Definește o clasă numită BookingPlugin care conține metode pentru rezervarea hotelurilor și a zborurilor.

Metoda de Rezervare Hotel:

- `@kernel_function(description="booking hotel")`: Un decorator care descrie funcția ca fiind o funcție kernel pentru rezervarea hotelurilor.
- `def booking_hotel(self, query: Annotated[str, "The name of the city"], check_in_date: Annotated[str, "Hotel Check-in Time"], check_out_date: Annotated[str, "Hotel Check-out Time"]) -> Annotated[str, "Return the result of booking hotel information"]:`: Definește o metodă pentru rezervarea hotelurilor cu parametri și tip de returnare adnotate.

Metoda construiește un dicționar de parametri pentru cererea de rezervare a hotelului și trimite o cerere GET către API-ul SERP. Verifică statusul răspunsului și returnează proprietățile hotelului dacă cererea a fost un succes sau None dacă cererea a eșuat.

Metoda de Rezervare Zbor:

- `@kernel_function(description="booking flight")`: Un decorator care descrie funcția ca fiind o funcție kernel pentru rezervarea zborurilor.
- `def booking_flight(self, origin: Annotated[str, "The name of Departure"], destination: Annotated[str, "The name of Destination"], outbound_date: Annotated[str, "The date of outbound"], return_date: Annotated[str, "The date of Return_date"]) -> Annotated[str, "Return the result of booking flight information"]:`: Definește o metodă pentru rezervarea zborurilor cu parametri și tip de returnare adnotate.

Metoda construiește dicționare de parametri pentru cererile de zbor dus și întors și trimite cereri GET către API-ul SERP. Verifică statusul răspunsului și adaugă informațiile despre zbor în șirul rezultat dacă cererea a fost un succes sau afișează un mesaj de eroare dacă cererea a eșuat. Metoda returnează șirul rezultat care conține informațiile despre zbor.


In [ ]:
import requests

from typing import Annotated

from semantic_kernel.functions import kernel_function

# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""

    @kernel_function(description="booking hotel")
    def booking_hotel(
        self, 
        query: Annotated[str, "The name of the city"], 
        check_in_date: Annotated[str, "Hotel Check-in Time"], 
        check_out_date: Annotated[str, "Hotel Check-out Time"],
    ) -> Annotated[str, "Return the result of booking hotel information"]:
        """
        Function to book a hotel.
        Parameters:
        - query: The name of the city
        - check_in_date: Hotel Check-in Time
        - check_out_date: Hotel Check-out Time
        Returns:
        - The result of booking hotel information
        """

        # Define the parameters for the hotel booking request
        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "1",
            "currency": "GBP",
            "gl": "uk",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request to the SERP API
        response = requests.get(BASE_URL, params=params)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the response content as JSON
            response = response.json()
            # Return the properties from the response
            return response["properties"]
        else:
            # Return None if the request failed
            return None

    @kernel_function(description="booking flight")
    def booking_flight(
        self, 
        origin: Annotated[str, "The name of Departure"], 
        destination: Annotated[str, "The name of Destination"], 
        outbound_date: Annotated[str, "The date of outbound"], 
        return_date: Annotated[str, "The date of Return_date"],
    ) -> Annotated[str, "Return the result of booking flight information"]:
        """
        Function to book a flight.
        Parameters:
        - origin: The name of Departure
        - destination: The name of Destination
        - outbound_date: The date of outbound
        - return_date: The date of Return_date
        - airline: The preferred airline carrier
        - hotel_brand: The preferred hotel brand
        Returns:
        - The result of booking flight information
        """
        
        # Define the parameters for the outbound flight request
        go_params = {
            "engine": "google_flights",
            "departure_id": "destination",
            "arrival_id": "origin",
            "outbound_date": "outbound_date",
            "return_date": "return_date",
            "currency": "GBP",
            "hl": "en",
            "airline": "airline",
            "hotel_brand": "hotel_brand",
            "api_key": "SERP_API_KEY"
        }
 
        print(go_params)

        # Send the GET request for the outbound flight
        go_response = requests.get(BASE_URL, params=go_params)

        # Initialize the result string
        result = ''

        # Check if the outbound flight request was successful
        if go_response.status_code == 200:
            # Parse the response content as JSON
            response = go_response.json()
            # Append the outbound flight information to the result
            result += "# outbound \n " + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Define the parameters for the return flight request
        back_params = {
            #"engine": "google_flights",
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": outbound_date,
            "return_date": return_date,
            "currency": "GBP",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request for the return flight
        back_response = requests.get(BASE_URL, params=back_params)

        # Check if the return flight request was successful
        if back_response.status_code == 200:
            # Parse the response content as JSON
            response = back_response.json()
            # Append the return flight information to the result
            result += "\n # return \n" + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Print the result
        print(result)

        # Return the result
        return result


# Explicație:
Declarații de Import: Importă modulele necesare pentru acreditivele Azure, agentul AI, conținutul mesajelor de chat, rolul autorului și decoratorul funcției kernel.

Manager de Context Asincron: `async with (DefaultAzureCredential() as creds, AzureAIAgent.create_client(credential=creds, conn_str="...") as client,)`: Configurează un manager de context asincron pentru a gestiona acreditivele Azure și pentru a crea un client pentru agentul AI.

Nume și Instrucțiuni pentru Agent: 
- `AGENT_NAME = "BookingAgent"`: Definește numele agentului.
- `AGENT_INSTRUCTIONS = """..."""`: Oferă instrucțiuni detaliate pentru agent despre cum să gestioneze cererile de rezervare.

Crearea Definiției Agentului: `agent_definition = await client.agents.create_agent(...)`: Creează o definiție a agentului cu modelul, numele și instrucțiunile specificate.

Crearea Agentului AzureAI: `agent = AzureAIAgent(...)`: Creează un agent AzureAI utilizând clientul, definiția agentului și pluginul definit.

Crearea unui Thread: `thread: AzureAIAgentThread | None = None`: Creează un thread pentru agent. Nu este necesar să creezi mai întâi un thread - dacă este furnizată valoarea `None`, un thread nou va fi creat în timpul primei invocări și va fi returnat ca parte a răspunsului.

Intrări Utilizator: `user_inputs = ["..."]`: Definește o listă de intrări ale utilizatorului pe care agentul trebuie să le proceseze.

În blocul `finally`, șterge thread-ul și agentul pentru a elibera resursele.


# Autentificare

Clasa `DefaultAzureCredential` face parte din Azure SDK pentru Python. Aceasta oferă o metodă implicită de autentificare cu serviciile Azure. Încearcă să se autentifice folosind mai multe metode într-o ordine specifică, cum ar fi variabilele de mediu, identitatea gestionată și acreditivele Azure CLI.

Operațiuni asincrone: Modulul aio indică faptul că clasa DefaultAzureCredential suportă operațiuni asincrone. Acest lucru înseamnă că o poți utiliza cu asyncio pentru a efectua cereri de autentificare fără blocare.


In [ ]:
# Import necessary modules
from azure.identity.aio import DefaultAzureCredential
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings, AzureAIAgentThread

ai_agent_settings = AzureAIAgentSettings.create()

# Azure AI Setting
async with (
     DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
    ) as client,
):    
    
    # Define the agent's name and instructions
    AGENT_NAME = "BookingAgent"
    AGENT_INSTRUCTIONS = """
    You are a booking agent, help me to book flights or hotels.

    Thought: Understand the user's intention and confirm whether to use the reservation system to complete the task.

    Action:
    - If booking a flight, convert the departure name and destination name into airport codes.
    - If booking a hotel or flight, use the corresponding API to call. Ensure that the necessary parameters are available. If any parameters are missing, use default values or assumptions to proceed.
    - If it is not a hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
    - For flight bookings, separate the outbound and return contents and list them in the order of Departure_airport Name | Airline | Flight Number | Departure Time | Arrival_airport Name | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
    - For hotel bookings, list them in the order of Properties Name | Properties description | check_in_time | check_out_time | prices | nearby_places | hotel_class | gps_coordinates.
    """

    # Create agent definition with the specified model, name, and instructions
    agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=AGENT_NAME,
        instructions=AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent using the client and agent definition
    agent = AzureAIAgent(
        client=client,
        definition=agent_definition,
        plugins=[BookingPlugin()]
    )

    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: AzureAIAgentThread | None = None

    # This is your prompt for the activity or task you want to complete 
    # Define user inputs for the agent to process we have provided some example prompts to test and validate 
    user_inputs = [
        # "Can you tell me the round-trip air ticket from  London to New York JFK aiport, the departure time is February 17, 2025, and the return time is February 23, 2025"
        # "Book a hotel in New York from Feb 20,2025 to Feb 24,2025"
        "Help me book flight tickets and hotel for the following trip London Heathrow LHR Feb 20th 2025 to New York JFK returning Feb 27th 2025 flying economy with British Airways only. I want a stay in a Hilton hotel in New York please provide costs for the flight and hotel"
        # "I have a business trip from London LHR to New York JFK on Feb 20th 2025 to Feb 27th 2025, can you help me to book a hotel and flight tickets"
    ]

    try:
        # Process each user input
        for user_input in user_inputs:
            print(f"# User: '{user_input}'")
            # Get the agent's response for the specified thread
            response = await agent.get_response(
                messages=user_input,
                thread=thread,
            )
            thread = response.thread
            # Print the agent's response
            print(f"{response.name}: '{response.content}'")
    finally:
        # Clean up by deleting the thread and agent
        await thread.delete() if thread else None
        await client.agents.delete_agent(agent.id)


---

**Declinarea responsabilității**:  
Acest document a fost tradus utilizând serviciul de traducere AI [Co-op Translator](https://github.com/Azure/co-op-translator). Deși depunem eforturi pentru a asigura acuratețea, vă rugăm să aveți în vedere că traducerile automate pot conține erori sau inexactități. Documentul original în limba sa nativă ar trebui considerat sursa autoritară. Pentru informații critice, se recomandă traducerea realizată de un profesionist uman. Nu ne asumăm răspunderea pentru eventualele neînțelegeri sau interpretări greșite care pot apărea din utilizarea acestei traduceri.
